In [1]:
from dotenv import load_dotenv
import textwrap
import os
import pandas as pd
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
import openai

#load our environment
load_dotenv()

ENV_NAM = os.getenv("ENV_NAME")

#configure Azure OpenAI Settings
#os.environ["OPENAI_API_TYPE"] = os.getenv"azure"
#os.environ["OPENAI_API_BASE"] = os.getenv"YOUR_OPENAI_ENDPOINT"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
#os.environ["OPENAI_API_VERSION"] = os.getenv"2023-05-15"
model: str = "text-embedding-ada-002"
vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
vector_store_key: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")

from azure.search.documents.indexes.models import (
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    ScoringProfile,
    TextWeights,
)

embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=model, chunk_size=1)
embedding_function = embeddings.embed_query

In [2]:
def wrap_text(text: str, width: int = 120) -> str:
    """Wrap text to a specified width."""
    return '\n'.join(textwrap.wrap(text, width))


In [3]:
def search(query: str):

    index_name: str = "centriqe-vector-demo"

    vector_store: AzureSearch = AzureSearch(
        azure_search_endpoint=vector_store_address,
        azure_search_key=vector_store_key,
        index_name=index_name,
        embedding_function=embedding_function,
    )

    # Perform a similarity search
    docs = vector_store.similarity_search(
        query=query,
        k=3,
        search_type="similarity",
    )
    
    return docs[0].page_content

In [12]:
#sample query to set the context
query=input()
context=search(query)

In [5]:
with open('context.txt', 'a') as fp:
    fp.write(str(context))

In [13]:
system = f'You are a simple QA chatbot.  Use the following context to provde responses to questions: \n\n CONTEXT: \n\n{context}'

In [7]:
with open('system.txt', 'a') as fp:
    fp.write(str(system))

In [14]:
question=query
messages=[
        {"role": "system", "content": f'{system}'},
        {"role": "user", "content": f'{query}'}
    ]

In [9]:
import json

with open('messages.json', 'a') as fp:
    fp.write(json.dumps(messages))

In [15]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    messages=messages,
    temperature=1, #gives more idea generation and a less consistent story
    stream=True,
)

In [16]:

# create variables to collect the stream of chunks
collected_messages = []
# iterate through the stream of events
for chunk in response:
    chunk_message = chunk['choices'][0]['delta']  # extract the message
    collected_messages.append(chunk_message)  # save the message
    #print(f"Message received: {chunk_message}")  # print the delay and text

full_reply_content = ''.join([m.get('content', '') for m in collected_messages])
print(wrap_text(full_reply_content))

Mosquera estimated that it would take a minimum of three years to change the culture and create a new system at MSD.
